# Let's Fetch **The Price of Daily Essentials in India Today**

In [18]:
import pandas as pd
import json, time, random
from datetime import datetime
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [19]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

In [20]:
source_urls = "sources.csv"

urls = []
with open(source_urls, 'r') as file:
    urls = [url.strip() for url in file if url.strip()]
    urls = [url.split('?')[0] for url in urls]
    seen = set()
    urls = [url for url in urls if not (url in seen or seen.add(url))]

df_products = []
current_date = datetime.now().strftime("%Y-%m-%d")

for i, url in enumerate(urls, 1):
    try:
        print(f"[{i}/{len(urls)}] Processing: {url}")
        driver.get(url)
        
        # Random sleep between 0.5 and 3.0 seconds, rounded to 1 decimal place
        sleep_time = round(random.uniform(0.5, 3.0), 1)
        time.sleep(sleep_time)

        jace = driver.find_element(By.ID, "__NEXT_DATA__").get_attribute("innerHTML")
        jace_data = json.loads(jace)['props']['pageProps']['productDetails']['children'][0]

        product = {
            "Date": current_date,
            "Category": jace_data['category']['tlc_name'],
            "Brand": jace_data['brand']['name'],
            "Product": jace_data['desc'],
            "Quantity": jace_data['w'],
            "Price-MRP": float(jace_data['pricing']['discount']['mrp']),
            "Price": float(jace_data['pricing']['discount']['prim_price']['sp'])
        }

        df_products.append(product)

    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}\n")

driver.quit()

[1/7] Processing: https://www.bigbasket.com/pd/40161155/farm-made-free-range-eggs-24-pcs/
[2/7] Processing: https://www.bigbasket.com/pd/40001002/surf-excel-quick-wash-detergent-powder-2-kg/
[3/7] Processing: https://www.bigbasket.com/pd/271205/id-fresho-idly-dosa-batter-1-kg/
[4/7] Processing: https://www.bigbasket.com/pd/30003294/id-fresho-malabar-parotaparatha-350-g-pouch/
[5/7] Processing: https://www.bigbasket.com/pd/40092421/kelloggs-corn-flakes-with-real-almond-honey-1-kg/
[6/7] Processing: https://www.bigbasket.com/pd/30008323/gillette-fusion-power-shaving-razor-blades-cartridge-8-pcs/
[7/7] Processing: https://www.bigbasket.com/pd/10001001/fresho-chicken-breast-boneless-antibiotic-residue-free-1-kg/


In [22]:
df_products = pd.DataFrame(df_products)
df_products['Units'] = df_products['Quantity'].str.split().str[-1]
df_products['Quantity'] = df_products['Quantity'].str.split().str[0]
df_products = df_products[['Date', 'Category', 'Brand', 'Product', 'Quantity', 'Units', 'Price']]

In [23]:
print("\nPrices of Daily Essentials:", "\n", "-"*50)
display(df_products)


Prices of Daily Essentials: 
 --------------------------------------------------


,Date,Category,Brand,Product,Quantity,Units,Price
0,2025-07-26,"Eggs, Meat & Fish",Farm Made,Free Range Eggs,24,pcs,540.55
1,2025-07-26,Cleaning & Household,Surf Excel,Quick Wash Detergent Powder,2,kg,420.00
2,2025-07-26,Snacks & Branded Foods,iD fresho!,Idly & Dosa Batter,1,kg,99.00
3,2025-07-26,Snacks & Branded Foods,iD fresho!,Malabar Parota - No Added Preservatives,400,g,108.90
4,2025-07-26,Snacks & Branded Foods,Kellogg's,Corn Flakes - With Real Almond & Honey,1,kg,395.57
5,2025-07-26,Beauty & Hygiene,Gillette,Fusion Power Blades for men Perfect Shave and...,8,pcs,2231.00
6,2025-07-26,"Eggs, Meat & Fish",fresho!,Chicken Breast Boneless,1,kg,469.00
